In [1]:
from pathlib import Path

In [2]:
save_dir = Path("../") / "BayesLogReg" / "bayes_logistic" / "pretrained"

In [3]:
import pickle

In [4]:
with open(save_dir / 'x_extract.pickle', 'rb') as f_x, open(save_dir / 'y_extract.pickle', 'rb') as f_y:
    # Pickle the 'data' dictionary using the highest protocol available.
    x_extract = pickle.load(f_x)
    y_extract = pickle.load(f_y)

In [5]:
print(y_extract['train'][:20])

[5. 0. 4. 1. 9. 2. 1. 3. 1. 4. 3. 5. 3. 6. 1. 7. 2. 8. 6. 9.]


In [6]:
print(y_extract['val'][:20])

[7. 2. 1. 0. 4. 1. 4. 9. 5. 9. 0. 6. 9. 0. 1. 5. 9. 7. 3. 4.]


In [7]:
import numpy as np

In [8]:
y_extract

{'train': array([5., 0., 4., ..., 5., 6., 8.]),
 'val': array([7., 2., 1., ..., 4., 5., 6.])}

In [9]:
x_extract

{'train': array([[22.48517418, 14.61251354, 27.79730988, ..., 22.38022995,
         24.60155106, 20.74303436],
        [30.97985077, 17.52405548, 24.48457146, ..., 20.76237869,
         25.33365631, 19.87883377],
        [19.80603027, 20.39133835, 17.72837067, ..., 16.84418106,
         19.95091057, 19.42064476],
        ...,
        [17.22813606, 12.17297745, 23.73967361, ..., 19.77940559,
         19.77432823, 19.95170784],
        [17.67393494, 14.05826092, 17.64319038, ..., 13.33616924,
         16.925951  , 14.98935986],
        [17.06033897, 13.15117264, 16.17965317, ..., 17.99224663,
         19.5974369 , 18.54821396]]),
 'val': array([[12.32572365, 15.35899067, 12.46453094, ..., 23.22957993,
         10.54736519, 18.8499279 ],
        [22.80735397, 20.179739  , 26.58662224, ..., 14.57098484,
         22.49274445, 19.51333618],
        [ 9.86898041, 17.40467262,  8.67834949, ..., 12.12457466,
         10.97650528, 11.81525993],
        ...,
        [25.49707222, 16.16849518, 19.

In [10]:
x_extract['train'].shape

(60000, 10)

In [11]:
y_extract['train'].shape

(60000,)

In [12]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_extract['train'] = scaler.fit_transform(x_extract['train'])
x_extract['val'] = scaler.transform(x_extract['val'])

In [13]:
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.nn.functional as F           # layers, activations and more
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.
from torch.jit import script, trace       # hybrid frontend decorator and tracing jit

In [14]:
class Linear_2L_KFRA(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()

        self.input_dim = input_dim
        self.output_dim = output_dim

        self.fc3 = nn.Linear(self.input_dim + 1, self.output_dim, bias=False)

        # choose your non linearity
        self.a2 = None
        self.h3 = None

    def forward(self, x):
        one = x.new(x.shape[0], 1).fill_(1)
        self.a2 = x
        self.a2 = torch.cat((self.a2.data, one), dim=1)
        self.h3 = self.fc3(self.a2)

        return self.h3

    def sample_predict(self, x, Nsamples, Qinv3, HHinv3, MAP3):
        predictions = x.data.new(Nsamples, x.shape[0], self.output_dim)
        one = x.new(x.shape[0], 1).fill_(1)
        self.a2 = x
        self.a2 = torch.cat((self.a2.data, one), dim=1)
        w3_list = MAP3.data.new(Nsamples, MAP3.size(0), MAP3.size(1))
        for i in range(Nsamples):
            w3 = sample_K_laplace_MN(MAP3, Qinv3, HHinv3)
            y = torch.matmul(self.a2, torch.t(w3))
            predictions[i] = y
            w3_list[i] = w3

        return predictions, w3_list       

In [15]:
def softmax_CE_preact_hessian(last_layer_acts):
    side = last_layer_acts.shape[1]
    I = torch.eye(side).type(torch.ByteTensor)
    # for i != j    H = -ai * aj -- Note that these are activations not pre-activations
    Hl = - last_layer_acts.unsqueeze(1) * last_layer_acts.unsqueeze(2)
    # for i == j    H = ai * (1 - ai)
    Hl[:, I] = last_layer_acts*(1-last_layer_acts)
    return Hl

def layer_act_hessian_recurse(prev_hessian, prev_weights, layer_pre_acts):
    
    newside = layer_pre_acts.shape[1]
    batch_size = layer_pre_acts.shape[0]
    I = torch.eye(newside).type(torch.ByteTensor) # .unsqueeze(0).expand([batch_size, -1, -1])
    
#     print(d_act(layer_pre_acts).unsqueeze(1).shape, I.shape)
    B = prev_weights.data.new(batch_size, newside, newside).fill_(0)
    B[:, I] = (layer_pre_acts > 0).type(B.type()) # d_act(layer_pre_acts)
    D = prev_weights.data.new(batch_size, newside, newside).fill_(0) # is just 0 for a piecewise linear
#     D[:, I] = dd_act(layer_pre_acts) * act_grads

    Hl = torch.bmm(torch.t(prev_weights).unsqueeze(0).expand([batch_size, -1, -1]), prev_hessian)    
    Hl = torch.bmm(Hl, prev_weights.unsqueeze(0).expand([batch_size, -1, -1]))
    Hl = torch.bmm(B, Hl)
    Hl = torch.matmul(Hl, B)
    Hl = Hl + D
    
    return Hl

def chol_scale_invert_kron_factor(factor, prior_scale, data_scale, upper=False):
    
    scaled_factor = data_scale * factor + prior_scale * torch.eye(factor.shape[0]).type(factor.type())
    inv_factor = torch.inverse(scaled_factor)
    chol_inv_factor = torch.cholesky(inv_factor, upper=upper)
    return chol_inv_factor

def sample_K_laplace_MN(MAP, upper_Qinv, lower_HHinv):
    # H = Qi (kron) HHi
    # sample isotropic unit variance mtrix normal
    Z = MAP.data.new(MAP.size()).normal_(mean=0, std=1)
    # AAT = HHi
#     A = torch.cholesky(HHinv, upper=False)
    # BTB = Qi
#     B = torch.cholesky(Qinv, upper=True)
    all_mtx_sample = MAP + torch.matmul(torch.matmul(lower_HHinv, Z), upper_Qinv)
    
    return all_mtx_sample

In [16]:
class BaseNet(object):
    def __init__(self):
        cprint('c', '\nNet:')

    def get_nb_parameters(self):
        return np.sum(p.numel() for p in self.model.parameters())

    def set_mode_train(self, train=True):
        if train:
            self.model.train()
        else:
            self.model.eval()

    def update_lr(self, epoch, gamma=0.99):
        self.epoch += 1
        if self.schedule is not None:
            if len(self.schedule) == 0 or epoch in self.schedule:
                self.lr *= gamma
                print('learning rate: %f  (%d)\n' % self.lr, epoch)
                for param_group in self.optimizer.param_groups:
                    param_group['lr'] = self.lr

    def save(self, filename):
        cprint('c', 'Writting %s\n' % filename)
        torch.save({
            'epoch': self.epoch,
            'lr': self.lr,
            'model': self.model,
            'optimizer': self.optimizer}, filename)

    def load(self, filename):
        cprint('c', 'Reading %s\n' % filename)
        state_dict = torch.load(filename)
        self.epoch = state_dict['epoch']
        self.lr = state_dict['lr']
        self.model = state_dict['model']
        self.optimizer = state_dict['optimizer']
        print('  restoring epoch: %d, lr: %f' % (self.epoch, self.lr))
        return self.epoch

In [17]:
class KBayes_Net(BaseNet):
    eps = 1e-6

    def __init__(self, lr=1e-3, side_in=10, cuda=True, classes=10, n_hid=1200, batch_size=128, prior_sig=0):
        super(KBayes_Net, self).__init__()
        cprint('y', ' Creating Net!! ')
        self.lr = lr
        self.cuda = cuda
        self.side_in=side_in
        self.prior_sig = prior_sig
        self.classes = classes
        self.batch_size = batch_size
        self.create_net()
        self.create_opt()
        self.epoch = 0
    def create_net(self):
        torch.manual_seed(42)
        if self.cuda:
            torch.cuda.manual_seed(42)

        self.model = Linear_2L_KFRA(input_dim=self.side_in, output_dim=self.classes)
        if self.cuda:
            self.model.cuda()
#             cudnn.benchmark = True

        print('    Total params: %.2fM' % (self.get_nb_parameters() / 1000000.0))

    def create_opt(self):
        self.optimizer = torch.optim.SGD(self.model.parameters(), lr=self.lr, momentum=0.5, weight_decay=1/self.prior_sig**2)

    def fit(self, x, y):
        x, y = x.cuda(), y.long().cuda()

        self.optimizer.zero_grad()

        out = self.model(x)
        loss = F.cross_entropy(out, y, reduction='sum')
            
        loss.backward()
        self.optimizer.step()

        pred = out.data.max(dim=1, keepdim=False)[1]  # get the index of the max log-probability
        err = pred.ne(y.data).sum()

        return loss.data, err

    def eval(self, x, y, train=False):
        x, y = x.cuda(), y.long().cuda()

        out = self.model(x)

        loss = F.cross_entropy(out, y, reduction='sum')
        probs = F.softmax(out, dim=1).data.cpu()

        pred = out.data.max(dim=1, keepdim=False)[1]  # get the index of the max log-probability
        err = pred.ne(y.data).sum()

        return loss.data, err, probs
    
    
    def get_K_laplace_params(self, trainloader):
        self.model.eval()

        it_counter = 0
        cum_HH3 = self.model.fc3.weight.data.new(self.model.output_dim, self.model.output_dim).fill_(0)
        cum_Q3 = self.model.fc3.weight.data.new(self.model.input_dim+1, self.model.input_dim+1).fill_(0)

        # Forward pass

        for x, y in trainloader:

            x, y = x.cuda(), y.long().cuda()

            self.optimizer.zero_grad()

            out = self.model(x)
            out_act = F.softmax(out, dim=1)

            #     ------------------------------------------------------------------
            HH3 = softmax_CE_preact_hessian(out_act.data)
            cum_HH3 += HH3.sum(dim=0)

            Q3 = torch.bmm(self.model.a2.data.unsqueeze(2), self.model.a2.data.unsqueeze(1))
            cum_Q3 += Q3.sum(dim=0)
            #     ------------------------------------------------------------------
            it_counter += x.shape[0]
            print(it_counter)

        EHH3 = cum_HH3 / it_counter

        EQ3 = cum_Q3 / it_counter

        MAP3 = self.model.fc3.weight.data
        
        return EQ3, EHH3, MAP3
    
    def sample_eval(self, x, y, Nsamples, scale_inv_EQ3, scale_inv_EHH3, MAP3):
        x, y = x.cuda(), y.long().cuda()

        out, w3_list = self.model.sample_predict(x, Nsamples, scale_inv_EQ3, scale_inv_EHH3, MAP3)
        
        mean_out = F.softmax(out, dim=2).mean(dim=0, keepdim=False)
        probs = mean_out.data.cpu()

        log_mean_probs_out = torch.log(mean_out)
        loss = F.nll_loss(log_mean_probs_out, y, reduction='sum')

        pred = mean_out.data.max(dim=1, keepdim=False)[1]  # get the index of the max log-probability
        err = pred.ne(y.data).sum()

        return loss.data, err, probs, w3_list

In [18]:
import os
import sys
def mkdir(paths):
    if not isinstance(paths, (list, tuple)):
        paths = [paths]
    for path in paths:
        if not os.path.isdir(path):
            os.makedirs(path)
  
def cprint(color, text, **kwargs):
    if color[0] == '*':
        pre_code = '1;'
        color = color[1:]
    else:
        pre_code = ''
    code = {
        'a': '30',
        'r': '31',
        'g': '32',
        'y': '33',
        'b': '34',
        'p': '35',
        'c': '36',
        'w': '37'
    }
    print("\x1b[%s%sm%s\x1b[0m" % (pre_code, code[color], text), **kwargs)
    sys.stdout.flush()

In [19]:
import time
import torch.utils.data
from torchvision import transforms, datasets
import matplotlib

models_dir = 'models_KLaplace_NN_MNIST'
results_dir = 'results_KLaplace_NN_MNIST'

mkdir(models_dir)
mkdir(results_dir)
# ------------------------------------------------------------------------------------------------------
# train config
batch_size = 10000
nb_epochs = 20 # We can do less iterations as this method has faster convergence

# ------------------------------------------------------------------------------------------------------
# dataset
cprint('c', '\nData:')


# load data

use_cuda = torch.cuda.is_available()

trainset = torch.utils.data.TensorDataset(torch.tensor(x_extract['train'], dtype=torch.float), 
                                          torch.tensor(y_extract['train'], dtype=torch.long))

valset = torch.utils.data.TensorDataset(torch.tensor(x_extract['val'], dtype=torch.float), 
                                         torch.tensor(y_extract['val'], dtype=torch.long))


if use_cuda:
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=3)
    valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=3)

else:
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory=False,
                                              num_workers=3)
    valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, pin_memory=False,
                                            num_workers=3)


## ---------------------------------------------------------------------------------------------------------------------
# net dims
cprint('c', '\nNetwork:')

lr = 1e-3
prior_sig = 1
########################################################################################
net = KBayes_Net(lr=lr, side_in=10, cuda=use_cuda, classes=10, batch_size=batch_size, prior_sig=prior_sig)

epoch = 0

## ---------------------------------------------------------------------------------------------------------------------
# train
cprint('c', '\nTrain:')

print('  init cost variables:')
cost_train = np.zeros(nb_epochs)
err_train = np.zeros(nb_epochs)

cost_dev = np.zeros(nb_epochs)
err_dev = np.zeros(nb_epochs)
best_err = np.inf


nb_its_dev = 1

tic0 = time.time()
for i in range(epoch, nb_epochs):
    
    net.set_mode_train(True)

    tic = time.time()
    nb_samples = 0

    for x, y in trainloader:
        cost, err = net.fit(x, y)

        err_train[i] += err
        cost_train[i] += cost
        nb_samples += len(x)

    cost_train[i] /= nb_samples
    err_train[i] /= nb_samples

    toc = time.time()
    net.epoch = i
    # ---- print
    print("it %d/%d, Jtr = %f, err = %f, " % (i, nb_epochs, cost_train[i], err_train[i]), end="")
    cprint('r', '   time: %f seconds\n' % (toc - tic))

    # ---- dev
    if i % nb_its_dev == 0:
        net.set_mode_train(False)
        nb_samples = 0
        for j, (x, y) in enumerate(valloader):

            cost, err, _ = net.eval(x, y)

            cost_dev[i] += cost
            err_dev[i] += err
            nb_samples += len(x)

        cost_dev[i] /= nb_samples
        err_dev[i] /= nb_samples

        cprint('g', '    Jdev = %f, err = %f\n' % (cost_dev[i], err_dev[i]))

        if err_dev[i] < best_err:
            best_err = err_dev[i]
            net.save(models_dir + '/theta_best.dat')

toc0 = time.time()
runtime_per_it = (toc0 - tic0) / float(nb_epochs)
cprint('r', '   average time: %f seconds\n' % runtime_per_it)


Data:

Network:

Net:
 Creating Net!! 
    Total params: 0.00M

Train:
  init cost variables:


/home/zl430/anaconda3/envs/pt-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  


it 0/20, Jtr = 4.029546, err = 0.605417,    time: 0.814029 seconds

    Jdev = 1.687383, err = 0.293500

Writting models_KLaplace_NN_MNIST/theta_best.dat



/home/zl430/anaconda3/envs/pt-gpu/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Linear_2L_KFRA. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


it 1/20, Jtr = 1.103602, err = 0.241850,    time: 0.432569 seconds

    Jdev = 0.615995, err = 0.175400

Writting models_KLaplace_NN_MNIST/theta_best.dat

it 2/20, Jtr = 0.544743, err = 0.152617,    time: 0.424394 seconds

    Jdev = 0.459049, err = 0.130500

Writting models_KLaplace_NN_MNIST/theta_best.dat

it 3/20, Jtr = 0.455274, err = 0.128350,    time: 0.381745 seconds

    Jdev = 0.418795, err = 0.118800

Writting models_KLaplace_NN_MNIST/theta_best.dat

it 4/20, Jtr = 0.425400, err = 0.120017,    time: 0.415982 seconds

    Jdev = 0.400490, err = 0.116400

Writting models_KLaplace_NN_MNIST/theta_best.dat

it 5/20, Jtr = 0.411549, err = 0.117250,    time: 0.459068 seconds

    Jdev = 0.392299, err = 0.114600

Writting models_KLaplace_NN_MNIST/theta_best.dat

it 6/20, Jtr = 0.402945, err = 0.115467,    time: 0.402985 seconds

    Jdev = 0.387726, err = 0.114300

Writting models_KLaplace_NN_MNIST/theta_best.dat

it 7/20, Jtr = 0.398804, err = 0.115083,    time: 0.470389 seconds

  

In [20]:
EQ3, EHH3, MAP3 = net.get_K_laplace_params(trainloader)
h_params = [EQ3, EHH3, MAP3]

with open(models_dir + '/block_hessian_params.pkl', 'wb') as f:
        pickle.dump(h_params, f)

10000
20000
30000
40000
50000
60000


In [21]:
EHH3.max()

tensor(0.0323, device='cuda:0')

In [22]:
MAP3.max()

tensor(8.0200, device='cuda:0')

In [23]:
EQ3.max()

tensor(1., device='cuda:0')

In [24]:
data_scale = np.sqrt(60000)

prior_prec = 1/prior_sig**2
prior_scale = np.sqrt(prior_prec)

scale_inv_EQ3 = chol_scale_invert_kron_factor(EQ3, prior_scale, data_scale, upper=True)
scale_inv_EHH3 = chol_scale_invert_kron_factor(EHH3, prior_scale, data_scale, upper=False)

## MAP

In [25]:
nb_samples = 0    

test_cost = 0  # Note that these are per sample
test_err = 0
test_predictions = np.zeros((10000, 10))

net.set_mode_train(False)

for j, (x, y) in enumerate(valloader):
    cost, err, probs = net.eval(x, y) # 

    test_cost += cost
    test_err += err.cpu().numpy()
    test_predictions[nb_samples:nb_samples+len(x), :] = probs.numpy()
    nb_samples += len(x)

test_err /= nb_samples
cprint('b', '    Loglike = %5.6f, err = %1.6f\n' % (-test_cost, test_err))

    Loglike = -3685.184082, err = 0.110400



In [26]:
MAP_test_predictions = test_predictions

## Laplace

In [27]:
nb_samples = 0    

test_cost = 0  # Note that these are per sample
test_err = 0
test_predictions = np.zeros((10000, 10))

net.set_mode_train(False)

Nsamples = 10000

for j, (x, y) in enumerate(valloader):
    cost, err, probs, w3_list = net.sample_eval(x, y, Nsamples, scale_inv_EQ3, scale_inv_EHH3, MAP3)

    test_cost += cost
    test_err += err.cpu().numpy()
    test_predictions[nb_samples:nb_samples+len(x), :] = probs.numpy()
    nb_samples += len(x)
    break

test_err /= nb_samples
cprint('b', '    Loglike = %5.6f, err = %1.8f\n' % (-test_cost, test_err))

    Loglike = -3685.733398, err = 0.11060000



In [28]:
Laplace_test_predictions = test_predictions

In [29]:
L_probs_idx = np.argmax(Laplace_test_predictions, axis=1)
M_probs_idx = np.argmax(MAP_test_predictions, axis=1)

In [30]:
L_probs = np.max(Laplace_test_predictions, axis=1)
M_probs = np.max(MAP_test_predictions, axis=1)

In [31]:
np.mean(L_probs - M_probs)

-0.000259663088619709